In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#load data into dataframe -- Working is subset of master with df['EVENT_TYPE'].isin(['Tornado','TORNADOES'])
data= pd.read_csv('../notebooks/NRI_Table_Counties.csv')

CRE Community resilience is the ability of a community to prepare for anticipated natural hazards, adapt to changing conditions, and withstand and recover rapidly from disruptions.

Social Vulnerability refers to the demographic and socioeconomic factors (such as poverty, lack of access to transportation, and crowded housing) that adversely affect communities that encounter hazards and other community-level stressors.



In [ ]:
df = data[[
    'COUNTY',
 'COUNTYTYPE',
 'COUNTYFIPS',
 'STCOFIPS',
 'POPULATION',  #Population (2020)
 'BUILDVALUE',  #Building Value ($)
 'AGRIVALUE', # Agriculture Value ($)
 'RESL_SCORE', #Community Resilience - Score Higher = More Resiliant
 'SOVI_SCORE',  #Social Vulnerability - Score == Higher =  More Vulnurable
]] 

df.fillna(value=0.0, inplace=True)

In [ ]:
# Devestation index taken as a multiple of Social Vulnerability and Community Resilience  
df['Devastation_Index'] = df.apply(lambda row: (row['SOVI_SCORE'] * (100- row['RESL_SCORE']))/10000, axis=1)

# Damage rate calculated per tornado damage compared by EF scale
# Fatality and injury rate derived from past tornadoes average : https://www.ulm.edu/~tfricker/pdf/13.pdf#:~:text=In%20addition%2C%20the%20per%2D%20capita%20fatality%20rate,(EF1)%20to%20a%20high%20of%204.2%%20(EF5).

columns = ['EF_Rating', 'Fatality_Rate', 'Injury_Rate', 'Damage_Rate']
data = [('EF0', 0.0018, 0.0781, 0.0000323 )
        , ('EF1', 0.0005, 0.0141 , 0.0003851)
        , ('EF2', 0.0007, 0.0145 , 0.0029625)
        , ('EF3', 0.0015, 0.0207 , 0.0217090)
        , ('EF4', 0.0021, 0.0307, 0.0749111)
        , ('EF5', 0.0062, 0.0417, 0.5941551 )
       ]

df_EF = pd.DataFrame(data, columns=columns)


In [ ]:
# Create Columns For Damage scores 
for index, row in df_EF.iterrows():
    #print( row['EF_Rating'], row['Fatality_Rate'], row['Injury_Rate'])
    df['Fatality'+'_'+row['EF_Rating']] = df['Devastation_Index'] * row['Fatality_Rate'] * df['POPULATION'] 
    df['Injury'+'_'+row['EF_Rating']] = df['Devastation_Index'] * row['Injury_Rate'] * df['POPULATION'] 
    df['Damage'+'_'+row['EF_Rating']] = df['Devastation_Index'] * row['Damage_Rate'] * df['BUILDVALUE']

In [ ]:
#Add to Csv file
df.to_csv('Tornado_Damage_By_County.csv')